In [11]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
from torch.autograd import Variable
from math import sqrt
import matplotlib.pyplot as plt
import numpy as np
import time

In [12]:
EPOCHS = 10
BATCH_SIZE = 10
LEARNING_RATE = 0.0003
TRAIN_DATAPATH = 'chest_xray\\train'
VAL_DATAPATH = 'chest_xray\\val'
TEST_DATAPATH = 'chest_xray\\test'
TRANSFORM_IMG = transforms.Compose([
    transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip(),
    transforms.Resize(256), transforms.CenterCrop(256), transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
    ])

train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATAPATH, transform=TRANSFORM_IMG)
train_data_loader = torch.utils.data.DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=4)
val_data = torchvision.datasets.ImageFolder(root=TEST_DATAPATH, transform=TRANSFORM_IMG)
val_data_loader = torch.utils.data.DataLoader(val_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=4)
test_data = torchvision.datasets.ImageFolder(root=TEST_DATAPATH, transform=TRANSFORM_IMG)
test_data_loader = torch.utils.data.DataLoader(test_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=4)


# def imshow(img):
#     img = img / 2 + 0.5     # unnormalize
#     npimg = img.numpy()
#     plt.figure(figsize=(20,20))
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))


# dataiter = iter(train_data_loader)
# images, labels = dataiter.next()
# # show images
# imshow(torchvision.utils.make_grid(images))


In [13]:
def weight_init(w):
    if w.__class__.__name__.find('Conv')!=-1:
        torch.nn.init.normal_(w.weight.data,0.0,0.2)

In [14]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)



In [23]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.reshape = nn.Sequential(
            nn.Linear(100, 3*256**2)
        
        )        
        self.discriminator = nn.Sequential(
            nn.Conv2d(3,128,kernel_size=3,stride=2,padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128,128,kernel_size=3,stride=2,padding=1),
            nn.LeakyReLU(0.2),
            Flatten(),
            nn.Dropout(0.4),
            nn.Linear(524288,3),
            nn.Softmax()
        )
        
    def forward(self,x):
        x = self.reshape(x)
        x = self.discriminant(x)
        return x
        
    

In [16]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
            
        self.generator = nn.Sequential(
            nn.ConvTranspose2d(100,512,kernel_size=4,stride=2),
            nn.LeakyReLU(0.2),
            nn.ConvTranspose2d(512,256,4,2,1,bias=False),
            nn.LeakyReLU(0.2),
            nn.ConvTranspose2d(256,128,kernel_size=(4,4),stride=2,bias=False),
            nn.LeakyReLU(0.2),
            nn.ConvTranspose2d(128,64,kernel_size=(4,4),stride=2,bias=False),
            nn.LeakyReLU(0.2),
            nn.ConvTranspose2d(64,3,kernel_size=(4,4),stride=2,bias=False),
            nn.Tanh()
        )
    
    def forward(self,x):
        x = self.generator(x)
        return x

In [17]:
print("Number of Training Examples: ", len(train_data))
print("Number of Test Examples: ", len(test_data))
print("Number of Valid Examples: ", len(val_data))
print("Detected Classes are: ", train_data.class_to_idx)
train_iter = iter(train_data_loader)
images, labels_ = train_iter.next()
print("Image Shape on Batch size = {} ".format(images.size()))

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)


Number of Training Examples:  5216
Number of Test Examples:  624
Number of Valid Examples:  624
Detected Classes are:  {'NORMAL': 0, 'PNEUMONIA': 1}
Image Shape on Batch size = torch.Size([10, 3, 256, 256]) 
cuda:0


In [18]:
dis = Discriminator()
gen = Generator()
lossAdv = nn.BCELoss()
lossAux = nn.CrossEntropyLoss()

In [25]:
dis.to(device)
gen.to(device)
lossAdv.to(device)
lossAux.to(device)
# _params = filter(lambda p: p.requires_grad, model.parameters())
# optimizer = optim.Adam(_params, lr=LEARNING_RATE)

RuntimeError: CUDA out of memory. Tried to allocate 3.13 GiB (GPU 0; 4.00 GiB total capacity; 0 bytes already allocated; 3.02 GiB free; 0 bytes cached)

In [ ]:
gen.apply(weight_init)
dis.apply(weight_init)

In [ ]:
optimizer_D = torch.optim.Adam(dis.parameters(), lr=3e-4,betas=(0.5,0.999))
optimizer_G = torch.optim.Adam(gen.parameters(), lr=3e-4,betas=(0.5,0.999))

In [ ]:
train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []
for epoch in range(EPOCHS):
    for i, data in enumerate(train_data_loader):
        
        images, labels = data[0].to(device),data[1].to(device)
        
        #Adverserial ground truths
        print(images.shape[0])
        valid = torch.cuda.FloatTensor(BATCH_SIZE,1).fill_(1.0).requires_grad_(False)
        fake = torch.cuda.FloatTensor(BATCH_SIZE,1).fill_(0.0).requires_grad_(False)
        fake_aux_gt = torch.cuda.LongTensor(BATCH_SIZE).fill_(2).requires_grad_(False)
        
        
        # Generator Train
        optimizer_G.zero_grad()
        
        z = torch.cuda.FloatTensor(np.random.normal(0,1,(BATCH_SIZE,100)))
        
        #Generate Images
        genimg = gen(z)
        
        #Loss measures generator's ability to fool dicriminator
        validity,_ = dis(genimg)
        g_loss = lossAdv(validity,valid)
        g_loss.backward()
        optimizer_G.step()
        
        
        optiimzer_D.zero_grad()
        
        #Loss for real images
        real_pred, real_aux = dis(images)
        d_real_loss = (lossAdv(real_pred,valid) + lossAux(real_aux,labels))/2
        
        #loss for fake images
        fake_pred, fake_aux = dis(genimg)
        d_fake_loss = (lossAdv(fake_pred,fake) + lossAux(fake_aux,fake_aux_gt))/2
                
        #total dis loss
        d_loss = (d_real_loss + d_fa)
        
        #Calculate discriminator Accuracy
        pred = np.concatenate([real_aux.data.cpu().numpy(), fake_aux_data.cpu().numpy()], axis=0)
        gt = np.concatenate([labels.data.cpu().numpy(), fake_aux_gt.data.cput().numpy()], axis=0)
        
        d_acc = np.mean(np.argmax(pred,axxis=1)==gt)
        
        d_loss.backward()
        optimizer_D.step()
        
        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %d%%] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), 100 * d_acc, g_loss.item())
        )
        
        
        

In [10]:
torch.cuda.empty_cache()